In [1]:
!pip install --quiet kagglehub bitsandbytes transformers accelerate

In [2]:
import os
import shutil
import kagglehub

In [3]:
# Set the path to your kaggle.json file (the API key)
os.environ['KAGGLE_CONFIG_DIR'] = "/teamspace/studios/this_studio/"

# Download Transformer Model

In [4]:
# Download latest version
cache_path = kagglehub.model_download("google/gemma/transformers/1.1-2b-it")

print("Path to model files:", cache_path)

100%|██████████| 1.53k/1.53k [00:00<00:00, 1.62MB/s]


100%|██████████| 13.2k/13.2k [00:00<00:00, 25.0MB/s]


100%|██████████| 132/132 [00:00<00:00, 91.4kB/s]


100%|██████████| 618/618 [00:00<00:00, 944kB/s]

 30%|██▉       | 5.00M/16.7M [00:00<00:00, 51.1MB/s]






100%|██████████| 39.7k/39.7k [00:00<00:00, 7.05MB/s]

100%|██████████| 16.7M/16.7M [00:00<00:00, 90.4MB/s]
100%|██████████| 4.04M/4.04M [00:00<00:00, 50.1MB/s]


100%|██████████| 64.0M/64.0M [00:00<00:00, 110MB/s]














































































































































































































100%|██████████| 4.61G/4.61G [00:23<00:00, 213MB/s]

Path to model files: /teamspace/studios/this_studio/.cache/kagglehub/models/google/gemma/transformers/1.1-2b-it/1


In [5]:
# Define the target directory where you want to move the model
target_path = os.path.expanduser("~/llm-models/gemma1_2b_model_transformer")

# Create the target directory if it does not exist
os.makedirs(target_path, exist_ok=True)

# Move the contents from the '1' folder (instead of moving the entire folder)
for item in os.listdir(cache_path):
    source = os.path.join(cache_path, item)
    destination = os.path.join(target_path, item)

    if os.path.isdir(source):
        shutil.move(source, destination)  # Move directories
    else:
        shutil.move(source, destination)  # Move files

# Optionally, remove the empty cache directory (if needed)
if not os.listdir(cache_path):
    os.rmdir(cache_path)  

print(f"Model files moved to: {target_path}")

Model files moved to: /teamspace/studios/this_studio/llm-models/gemma1_2b_model_transformer


# Convert to Pytorch

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [8]:
model_path = "/teamspace/studios/this_studio/llm-models/gemma1_2b_model_transformer"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Configure bitsandbytes for 4-bit quantization
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Load the model with device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",  # Ensures model parts are mapped to devices
    low_cpu_mem_usage=True  # Reduces CPU memory usage
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Save the model in PyTorch checkpoint (.ckpt) format
torch.save(model.state_dict(), 'sample1-gemma.ckpt')

# Convert Hugging Face Model to Pytorch

In [ ]:
# For loading model get access from the hugging face and load the token
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv(override=True)
hf_access_token = os.environ["hf_access_token"]

from huggingface_hub import login
login(token=hf_access_token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    torch_dtype=torch.bfloat16
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
# Save the model in PyTorch checkpoint (.ckpt) format
torch.save(model.state_dict(), 'sample-gemma.ckpt')